In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Importing Libraries

In [0]:
!pip install pydicom

In [0]:
!pip install h5py

In [0]:
import numpy as np
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import collections
from tqdm import tqdm_notebook as tqdm
from datetime import datetime

from math import ceil, floor, log
import cv2
from keras.models import model_from_json
import tensorflow as tf
import keras

import sys
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation, ZeroPadding2D, BatchNormalization, MaxPooling2D, Add, AveragePooling2D
from sklearn.model_selection import ShuffleSplit

###Declaring Variables

In [0]:
ALL_LABELS = 'gdrive/My Drive/stage_1_train.csv'
TEST_IMAGES_DIR = 'gdrive/My Drive/RSNA Sample 10 GB attempt 3/'
TEST_LABELS = 'gdrive/My Drive/test.csv'
TRAIN_IMAGES_DIR = 'gdrive/My Drive/RSNA Sample 10 GB attempt 3/'
TRAIN_LABELS = 'gdrive/My Drive/train.csv'
#For Saving
WEIGHTS_SAVE_DIR = 'gdrive/My Drive/Weights/'
WEIGHT_SAVING_NAME = 'DRN_Hidden_lr1e-6'
#For Loading
SHORTLIST_WEIGHTS_DIR = 'gdrive/My Drive/Weights/'
WEIGHT_LOAD_NAME = 'DRN_Hidden_lr1e-602.hdf5'

#Sampling (Between 0 and 1)
KEEP_PROB_0 = 1
KEEP_PROB_1 = 1

###Train Test Split (Needed to be run only once on a platform)

In [0]:
df = pd.read_csv(ALL_LABELS)
df["Image"] = df["ID"].str.slice(stop=12)
df["Diagnosis"] = df["ID"].str.slice(start=13)    
duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
df = df.drop(index=duplicates_to_remove)
df = df.reset_index(drop=True)
df = df.loc[:, ["Label", "Diagnosis", "Image"]]
df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)

In [0]:
###Caution for Google drive data (Avoid for other platforms)
root_path = 'gdrive/My Drive/RSNA Sample 10 GB attempt 3/'
import os
a = os.listdir(root_path)
a = [item.rstrip('.dcm') for item in a]
df = df.stack().reset_index()
df = df[df['Image'].isin(a)]
df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
len(df)

13062

In [0]:
from sklearn.model_selection import train_test_split
train, test  = train_test_split(df, test_size=0.2, random_state=1)
test = test.stack().reset_index()
test.insert(loc=0, column='ID', value=test['Image'].astype(str) + "_" + test['Diagnosis'])
test = test.drop(["Image", "Diagnosis"], axis=1)
train = train.stack().reset_index()
train.insert(loc=0, column='ID', value=train['Image'].astype(str) + "_" + train['Diagnosis'])
train = train.drop(["Image", "Diagnosis"], axis=1)
print(len(train),len(test))
test.to_csv(TEST_LABELS)
train.to_csv(TRAIN_LABELS)

62694 15678


###Windowing (BSB & Manual)

In [0]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(dcm, window_center, window_width):
    
    if (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)
    
    img = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)

    return img

def bsb_window(dcm):
    brain_img = window_image(dcm, 40, 80)
    subdural_img = window_image(dcm, 80, 200)
    soft_img = window_image(dcm, 40, 380)
    
    brain_img = (brain_img - 0) / 80
    subdural_img = (subdural_img - (-20)) / 200
    soft_img = (soft_img - (-150)) / 380
    bsb_img = np.array([brain_img, subdural_img, soft_img]).transpose(1,2,0)

    return bsb_img

In [0]:
# def manual_window(dcm, window_center, window_width):
#     img = window_image(dcm, window_center, window_width)
#     img = (img - (window_center - window_width/2) / 
#     manual_img = np.array([img, img, img]).transpose(1,2,0)

#     return manual_img

###Read images (Change Windowing here)

In [0]:
def _read(path, desired_size):
    """Will be used in DataGenerator"""
    
    dcm = pydicom.dcmread(path)
    
    try:
        img = bsb_window(dcm)
        #img = manual_window(dcm, 40, 80) # Specify window center, window_width here
    except:
        img = np.zeros(desired_size)
    
    
    img = cv2.resize(img, desired_size[:2], interpolation=cv2.INTER_LINEAR)
    
    return img


###Data generator

In [0]:
class DataGenerator(keras.utils.Sequence):

    def __init__(self, list_IDs, labels=None, batch_size=1, img_size=(512, 512, 1), 
                 img_dir=TRAIN_IMAGES_DIR, *args, **kwargs):

        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.img_size = img_size
        self.img_dir = img_dir
        self.on_epoch_end()

    def __len__(self):
        return int(ceil(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indices]
        
        if self.labels is not None:
            X, Y = self.__data_generation(list_IDs_temp)
            return X, Y
        else:
            X = self.__data_generation(list_IDs_temp)
            return X
        
    def on_epoch_end(self):
        
        
        if self.labels is not None: # for training phase we undersample and shuffle
            # keep probability of any=0 and any=1
            keep_prob = self.labels.iloc[:, 0].map({0: KEEP_PROB_0, 1: KEEP_PROB_1})
            keep = (keep_prob > np.random.rand(len(keep_prob)))
            self.indices = np.arange(len(self.list_IDs))#[keep]
            np.random.shuffle(self.indices)
        else:
            self.indices = np.arange(len(self.list_IDs))

    def __data_generation(self, list_IDs_temp):
        X = np.empty((self.batch_size, *self.img_size))
        
        if self.labels is not None: # training phase
            Y = np.empty((self.batch_size, 6), dtype=np.float32)
        
            for i, ID in enumerate(list_IDs_temp):
                X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
                Y[i,] = self.labels.loc[ID].values
        
            return X, Y
        
        else: # test phase
            for i, ID in enumerate(list_IDs_temp):
                X[i,] = _read(self.img_dir+ID+".dcm", self.img_size)
            
            return X

###Loss Function

In [0]:
from keras import backend as K


def _normalized_weighted_average(arr, weights=None):
    """
    A simple Keras implementation that mimics that of 
    numpy.average(), specifically for this competition
    """
    
    if weights is not None:
        scl = K.sum(weights)
        weights = K.expand_dims(weights, axis=1)
        return K.sum(K.dot(arr, weights), axis=1) / scl
    return K.mean(arr, axis=1)


def weighted_loss(y_true, y_pred):
    """
    Will be used as the metric in model.compile()
    ---------------------------------------------
    
    Similar to the custom loss function 'weighted_log_loss()' above
    but with normalized weights, which should be very similar 
    to the official competition metric:
        https://www.kaggle.com/kambarakun/lb-probe-weights-n-of-positives-scoring
    and hence:
        sklearn.metrics.log_loss with sample weights
    """
    
    class_weights = K.variable([2., 1., 1., 1., 1., 1.])
    
    eps = K.epsilon()
    
    y_pred = K.clip(y_pred, eps, 1.0-eps)

    loss = -(        y_true  * K.log(      y_pred)
            + (1.0 - y_true) * K.log(1.0 - y_pred))
    
    loss_samples = _normalized_weighted_average(loss, class_weights)
    
    return K.mean(loss_samples)

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc


###Model (Change hidden layers here) (Remove cross validation here)

In [0]:
def identity_block(X, f, filters, stage, block, dilation=1):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size = (1,1), strides = (1,1), \
                padding = 'valid', dilation_rate=dilation, name = conv_name_base+'2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), \
                padding = 'same', dilation_rate=dilation, name = conv_name_base+'2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), \
                padding = 'valid', dilation_rate=dilation, name = conv_name_base+'2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, stage, block, s=2, dilation=1):
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = Conv2D(filters = F1, kernel_size = (1,1), strides = (s,s), \
                padding = 'valid', dilation_rate=dilation, name = conv_name_base+'2a')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(filters = F2, kernel_size = (f,f), strides = (1,1), \
                padding = 'same', dilation_rate=dilation, name = conv_name_base+'2b')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), \
                padding = 'valid', dilation_rate=dilation, name = conv_name_base+'2c')(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1,1), strides=(s,s), padding='valid',\
                        name=conv_name_base + '1')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [0]:
    
class MyDeepModel:

    
  def __init__(self, engine, input_dims, batch_size=5, num_epochs=4, learning_rate=1e-3, 
                decay_rate=1.0, decay_steps=1, weights="imagenet", verbose=1):
        self.engine = engine
        self.input_dims = input_dims
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.learning_rate = learning_rate
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
        self.weights = weights
        self.verbose = verbose
        self._build()

  def _build(self):
        engine = self.engine(include_top=False, weights=self.weights, input_shape=self.input_dims, 
                            backend = keras.backend, layers = keras.layers,
                            models = keras.models, utils = keras.utils)
        
        # for layer in engine.layers[:80]:
        #   layer.trainable = False

        resnet_stage3 = engine.layers[80]

        X = convolutional_block(resnet_stage3.output, f=3, filters=[256, 256, 1024], stage=4, block='a', s=1, dilation=1)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b', dilation=2)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c', dilation=2)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d', dilation=2)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e', dilation=2)
        X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f', dilation=2)

        X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=1, dilation=2)
        X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b', dilation=4)
        X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c', dilation=4)

        X = Flatten()(X)
        out = keras.layers.Dense(6, activation="sigmoid", name='fc6')(X)

        # Create model

        self.model = keras.models.Model(inputs=engine.input, outputs=out, name='Dilated ResNet50')

        self.model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(), metrics=[weighted_loss, auc])
  

  def fit_model(self, train_df, valid_df):
      
    # callbacks
#For kaggle krnel os.chdir to working dir
        checkpointer = keras.callbacks.ModelCheckpoint(filepath=WEIGHTS_SAVE_DIR+WEIGHT_SAVING_NAME+'{epoch:02d}.hdf5' , verbose=1, save_weights_only=True, save_best_only=False, monitor='val_loss')
    #Change back to the input directory
        scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: self.learning_rate * pow(self.decay_rate, floor(epoch / self.decay_steps)))
        
        return self.model.fit_generator(
            DataGenerator(
                train_df.index, 
                train_df, 
                self.batch_size, 
                self.input_dims, 
                TRAIN_IMAGES_DIR
            ),
            validation_data = DataGenerator(
                valid_df.index, 
                valid_df, 
                self.batch_size, 
                self.input_dims, 
                TRAIN_IMAGES_DIR
            ),
            epochs=self.num_epochs,
            verbose=self.verbose,
            use_multiprocessing=True,
            workers=4,
            callbacks=[scheduler, checkpointer]
          )

  def save(self, path):
        self.model.save_weights(path)
  
  def load(self, path):
        self.model.load_weights(path)

  def predictions(self, test_df, batch_size_pred=8):
        self.test_df= test_df
        self.batch_size_pred = batch_size_pred
        test_preds = self.model.predict_generator(DataGenerator(self.test_df.index, None, self.batch_size_pred, self.input_dims, TEST_IMAGES_DIR), verbose=1)
        return test_preds


###Read and convert labels dataset to our format (Remove the duplicates when running on GCP)

In [0]:

def read_trainset(filename=TRAIN_LABELS):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
#    df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df

def read_testset(filename=TEST_LABELS):
    df = pd.read_csv(filename)
    df["Image"] = df["ID"].str.slice(stop=12)
    df["Diagnosis"] = df["ID"].str.slice(start=13)
    duplicates_to_remove = [
        1598538, 1598539, 1598540, 1598541, 1598542, 1598543,
        312468,  312469,  312470,  312471,  312472,  312473,
        2708700, 2708701, 2708702, 2708703, 2708704, 2708705,
        3032994, 3032995, 3032996, 3032997, 3032998, 3032999
    ]
    
    #df = df.drop(index=duplicates_to_remove)
    df = df.reset_index(drop=True)
    
    df = df.loc[:, ["Label", "Diagnosis", "Image"]]
    df = df.set_index(['Image', 'Diagnosis']).unstack(level=-1)
    
    return df


###Hyperparameters (Most Important Part) - Run this only when training

In [0]:
#To avoid tons of warning that are going to appear in training the model
import warnings
warnings.filterwarnings("ignore")

In [0]:
#Read train set
df = read_trainset()

# train set and validation set (Change number of splits)
ss = ShuffleSplit(n_splits=3, test_size=0.2, random_state=42).split(df.index)

train_idx, valid_idx = next(ss)

model = MyDeepModel(engine=ResNet50, input_dims=(224, 224, 3), batch_size=16, learning_rate=1e-6,
                    num_epochs=3, decay_rate=0.8, decay_steps=1, weights="imagenet", verbose=1)
model.load(SHORTLIST_WEIGHTS_DIR+WEIGHT_LOAD_NAME)
history = model.fit_model(df.iloc[train_idx], df.iloc[valid_idx])



Epoch 1/3
523/523 [==============================] - 1036s 2s/step - loss: 0.3759 - weighted_loss: 0.3972 - auc: 0.8481 - val_loss: 0.4709 - val_weighted_loss: 0.4994 - val_auc: 0.8430

Epoch 00001: saving model to gdrive/My Drive/McCombs/Fall2019/APM/Weights/DRN_Hidden_lr1e-601.hdf5
Epoch 2/3
523/523 [==============================] - 945s 2s/step - loss: 0.2986 - weighted_loss: 0.3172 - auc: 0.8556 - val_loss: 0.4709 - val_weighted_loss: 0.5001 - val_auc: 0.8629

Epoch 00002: saving model to gdrive/My Drive/McCombs/Fall2019/APM/Weights/DRN_Hidden_lr1e-602.hdf5
Epoch 3/3
523/523 [==============================] - 943s 2s/step - loss: 0.2528 - weighted_loss: 0.2685 - auc: 0.8714 - val_loss: 0.4730 - val_weighted_loss: 0.4992 - val_auc: 0.8782

Epoch 00003: saving model to gdrive/My Drive/McCombs/Fall2019/APM/Weights/DRN_Hidden_lr1e-603.hdf5
